In [48]:
import cv2
import torch
from facenet_pytorch import InceptionResnetV1, MTCNN
import numpy as np
import os
from PIL import Image
import numpy as np
import pyodbc
import io

In [43]:
# --- Inicializar el dispositivo: GPU si está disponible, si no, usar CPU ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- Inicializar el detector de rostros MTCNN ---
# Este modelo detecta y recorta automáticamente las caras
mtcnn = MTCNN(image_size=160, margin=0, device=device)

# --- Cargar el modelo de reconocimiento facial FaceNet (entrenado con VGGFace2) ---
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [44]:
#Lista para guardar las imagenes de las personas con acceso
authorized_embeddings = []
# Lista para guardar los Nombres de las personas con acceso
authorized_names = []

In [45]:
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 18 for SQL Server};"
    "SERVER=RICARDO;"  # Usa el nombre exacto de tu servidor
    "DATABASE=SucursalesMVC;"
    "Trusted_Connection=yes;"
    "Encrypt=yes;"
    "TrustServerCertificate=yes;"
)

cursor = conn.cursor()

In [46]:
# Leer imágenes desde la carpeta "faces"
path = "faces"
for file in os.listdir(path):
    img_path = os.path.join(path, file)
    img = Image.open(img_path)

    # Usar MTCNN para extraer y recortar el rostro de la imagen
    face = mtcnn(img)

    if face is not None:
        # Obtener el embedding (vector de 512 elementos)
        embedding = model(face.unsqueeze(0).to(device))  # Resultado: shape [1, 512]
        embedding = embedding.detach().cpu()[0]  # Remueve la dimensión batch (queda [512])

        # Agrega a lista de vectores autorizados
        authorized_embeddings.append(embedding)

        # Obtener nombre (sin extensión)
        nombre = os.path.splitext(file)[0]
        authorized_names.append(nombre)

        # Leer imagen original como bytes
        with open(img_path, "rb") as f:
            image_bytes = f.read()

        # Insertar en la base de datos (convertido a bytes)
        cursor.execute("INSERT INTO Embeddings (nombre, embedding, imagen) VALUES (?, ?, ?)",(nombre, embedding.numpy().tobytes(), image_bytes))
        conn.commit()


In [49]:
# Consulta: obtener las primeras 5 imágenes
cursor.execute("SELECT nombre, imagen FROM Embeddings WHERE imagen IS NOT NULL")

# Mostrar las primeras 5 imágenes
contador = 0
for nombre, image_bytes in cursor.fetchall():
    if image_bytes:
        image = Image.open(io.BytesIO(image_bytes))
        image.show(title=nombre)  # Muestra imagen en visor predeterminado
        print(f"Mostrando: {nombre}")
        
        contador += 1
        if contador >= 5:
            break

Mostrando: Bad Bunny
Mostrando: Ricardo


In [37]:
# Iniciar cámara
cap = cv2.VideoCapture(0)

In [38]:
# === LECTURA DE EMBEDDINGS DESDE LA BASE DE DATOS ===
cursor.execute("SELECT nombre, embedding FROM Embeddings")
rows = cursor.fetchall()

In [40]:
for nombre, embedding_bytes in rows:
    vector = np.frombuffer(embedding_bytes, dtype=np.float32)
    tensor = torch.tensor(vector)
    authorized_embeddings.append(tensor)
    authorized_names.append(nombre)

print(f"{len(authorized_embeddings)} embeddings cargados desde la base de datos.")

2 embeddings cargados desde la base de datos.


In [41]:
while True:
    ret, frame = cap.read()
    if not ret:
        break  # Salir si no hay frame

    # Convertir frame a formato PIL (RGB)
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Detectar rostro
    face = mtcnn(img)

    if face is not None:
        # Obtener embedding
        embedding = model(face.unsqueeze(0).to(device))
        embedding = embedding.detach().cpu().squeeze(0)

        # Comparar con embeddings autorizados
        distances = [torch.norm(e - embedding).item() for e in authorized_embeddings]
        min_dist = min(distances)
        index = distances.index(min_dist)

        # Clasificación
        name = "Desconocido"
        if min_dist < 0.9:
            name = authorized_names[index]

        # Mostrar resultados
        cv2.putText(frame, f"{name} ({min_dist:.2f})", (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (0, 255, 0) if name != "Desconocido" else (0, 0, 255), 2)

    # Mostrar video
    cv2.imshow("Reconocimiento Facial (PyTorch)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# === LIBERAR RECURSOS ===
cap.release()
cv2.destroyAllWindows()